In [ ]:
from functools import reduce
from fuzix.sims.pendulumcart import draw_world
from fuzix.sims.pendulumcart import simulate
from fuzix.utils import do_render_animation
from fuzix.utils import render_animation
from fuzix.utils import render_animation_iter
from fuzix.utils import memoize
from fuzix.utils import RendererWidget
from fuzix.utils import timed
from fuzix.utils import renderer_scope
from fuzix.draw_utils import draw_circle
from fuzix.draw_utils import draw_rect
from fuzix.draw_utils import draw_line
from IPython.display import display
from IPython.display import HTML
from IPython.display import Markdown
from ipywidgets import interact
from matplotlib import pyplot as plt, cm, animation, rc
from matplotlib.lines import Line2D
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
import daglet
import functools
import ipywidgets
import matplotlib
import matplotlib.patches as patches
import numbers
import numpy as np
import operator
import sympy as sp
from fuzix.scene import *

In [ ]:
%matplotlib inline
rc('animation', html='html5')
#rc('animation', html='jshtml')

sp.init_printing(use_unicode=True)
init_vprinting()

In [ ]:
GRAVITY = 10.


class XState(object):
    def __init__(self):
        pass
    
    def __repr__(self):
        return 'q<{:x}>'.format(hash(self))

    
class XHingeRod(object):
    def __init__(self, length=2., mass=1.):
        self.theta = XState()
        self.length = length
        self.mass = mass

    def get_pos(self, state_map):
        theta, _ = state_map[self.theta]
        return self.length * np.matrix([[np.sin(theta)], [-np.cos(theta)]])
    
    def get_vel(self, state_map):
        theta, dtheta = state_map[self.theta]
        return self.length * dtheta * np.matrix([[np.cos(theta)], [np.sin(theta)]])

    def get_accel_matrix(self, state_map):
        theta, dtheta = state_map[self.theta]
        return np.matrix([
            [
                self.length * np.cos(theta),
                -self.length * dtheta**2 * np.sin(theta),
            ],
            [
                self.length * np.sin(theta),
                self.length * dtheta**2 * np.cos(theta),
            ]
        ])
    
    def get_vel_perturbation(self, state_map, state):
        if state == self.theta:
            theta, _ = state_map[self.theta]
            return self.length * np.matrix([[np.cos(theta)], [np.sin(theta)]])
        else:
            return np.matrix([[0.], [0.]])
        
    def get_vel_pos_perturbation(self, state_map, state):  # god we need better names
        if state == self.theta:
            theta, dtheta = state_map[self.theta]
            return self.length * dtheta * np.matrix([[-np.sin(theta)], [np.cos(theta)]])
        else:
            return np.matrix([[0.], [0.]])    
    
    def get_vel_perturbation_deriv(self, state_map, state):
        if state == self.theta:
            theta, dtheta = state_map[self.theta]
            return self.length * dtheta * np.matrix([[-np.sin(theta)], [np.cos(theta)]])
        else:
            return np.matrix([[0.], [0.]])
    
    def get_potential_perturbation(self, state_map, state):
        if state == self.theta:
            theta, _ = state_map[self.theta]
            return self.length * np.sin(theta)
        else:
            return 0.
        
    def get_potential_vel_perturbation_deriv(self, state_map, state):
        return 0.
    

def get_onehot_vector(index, length):
    a = np.zeros(length)
    a[index] = 1.
    return np.matrix(a).T


def get_spacer_matrix(num_states, state_num):
    m = np.matrix(np.zeros([2, num_states + 1]))
    m[0, state_num] = 1.
    m[1, num_states] = 1.
    return m


def get_sub_lagrangian_row(objs, num_states, state_map, state, obj_num):
    mass = objs[obj_num].mass
    padder = get_onehot_vector(-1, num_states+1).T
    result = np.matrix(np.zeros([1, num_states + 1]))
    for i in range(obj_num + 1):
        for j in range(obj_num + 1):
            spacer = get_spacer_matrix(num_states, j)
            result += mass * objs[i].get_vel_perturbation(state_map, state).T * objs[j].get_accel_matrix(state_map) * spacer
            result += mass * objs[i].get_vel_perturbation_deriv(state_map, state).T * objs[j].get_vel(state_map) * padder
            result -= mass * objs[i].get_vel_pos_perturbation(state_map, state).T * objs[j].get_vel(state_map) * padder
        result -= mass * GRAVITY * objs[i].get_potential_perturbation(state_map, state) * padder
    return result
    

def solve_for_ddq(objs, states, state_map, dt=0.01):
    num_objs = len(objs)
    num_states = len(states)
    M = np.matrix(np.zeros([num_states, num_states + 1]))
    for state_num, state in enumerate(states):
        row = np.matrix(np.zeros([1, num_states + 1]))
        for obj_num in range(0, num_objs):            
            row += get_sub_lagrangian_row(objs, num_states, state_map, state, obj_num)
        expander = get_onehot_vector(state_num, num_states)
        M += expander * row

    A = M[:,:-1]
    b = -M[:,-1]
    ddq = np.linalg.inv(A) * b
    ddq_map = {}
    for state_num, state in enumerate(states):
        ddq_map[state] = ddq[state_num, 0]
    return ddq_map

    
def tick(objs, states, state_map, dt=0.01):
    ddq_map = solve_for_ddq(objs, states, state_map, dt)
    new_state_map = {}
    for state in states:
        ddq = ddq_map[state]
        q, dq = state_map[state]
        new_state_map[state] = [q + dq*dt, dq + ddq*dt]
    return new_state_map


def run(objs, states, initial_state_map, max_time, dt=0.01):
    state_map = initial_state_map
    state_maps = []
    for i in range(max_time):
        if i % 100 == 0:
            print('Frame {}'.format(i))
        state_map = tick(objs, states, state_map, dt)
        state_maps.append(state_map)
    return state_maps

In [ ]:
num_objs = 10
masses = [0.5] * (num_objs - 1) + [5.]
objs = [XHingeRod(mass=mass) for _, mass in zip(range(num_objs), masses)]
obj_states = [x.theta for x in objs]

spec_states = [State() for _ in range(num_objs)]
connector = Connector() | World()
for state, mass in zip(spec_states, masses):
    connector = connector | HingeRod(angle=state) | Ball(mass=mass)
scene = Scene([connector])


def get_ddq(obj, *args):
    thetas = args[:num_objs]
    dthetas = args[num_objs:]
    state_map = {obj2.theta: [theta, dtheta] for obj2, theta, dtheta in zip(objs, thetas, dthetas)}
    ddq_map = solve_for_ddq(obj_states, state_map)
    return ddq_map[obj.theta]

def convert_state_map(state_map):
    oldskool_state = {}
    for spec_state, obj_state in zip(spec_states, obj_states):
        oldskool_state[spec_state] = state_map[obj_state][0]
    return oldskool_state

def convert_state_maps(state_maps):
    return [convert_state_map(x) for x in state_maps]


scale = 0.25
initial_values = [np.pi - 0.9 - 0.1*i for i in range(num_objs)]
initial_state_map = {state: [value, 0.] for state, value in zip(obj_states, initial_values)}
scene.draw(state=convert_state_map(initial_state_map), scale=scale)
state_maps = [initial_state_map]

In [ ]:
state_maps += run(objs, obj_states, state_maps[-1], 100, dt=0.002)

In [ ]:
oldskool_states = convert_state_maps(state_maps)
show_scene_viewer(scene, params={}, states=oldskool_states, scale=scale)
show_renderer(scene, params={}, states=oldskool_states, scale=scale, sample_interval=18)

In [ ]:
l = Param()
k = Param()
x1 = State()
x2 = State()
angle1 = State()

world = Connector() | World()
track = world | CircleTrack()
block1 = track | Coaster(x=x1) | Ball()
block2 = track | Coaster(x=x2) | Ball()

scene = Scene([
    Connector([block1, block2]) | Spring(length=l, strength=k),
    block1 | HingeRod(angle=angle1) | Ball(),
])

In [ ]:
params = {}
initial_state = {}
scale = 0.8

@interact(
    x1_=(-8., 8.),
    x2_=(-8., 8.),
    l_=(3., 10.),
    k_=(0.5, 5.),
    angle1_=(-np.pi, np.pi),
)
def f(
    x1_=-0.5,
    x2_=-2.6,
    l_=6.,
    k_=2.,
    angle1_=-2.2,
):
    global params, initial_state
    params.update({
        l: l_,
        k: k_,
    })
    initial_state.update({
        x1: x1_,
        x2: x2_,
        angle1: angle1_,
    })
    scene.draw(params=params, state=initial_state, scale=scale)

In [ ]:
simulator = Simulator(scene)

In [ ]:
states = simulator.run(params, initial_state, max_time=800, dt=0.005)
show_scene_viewer(scene, params, states, scale=scale)
show_renderer(scene, params, states, scale=scale, sample_interval=6)

In [ ]:
#t = sp.Symbol('t')
#Q = [symbol for symbol, _ in scene.state_symbols]
#dQ = [q.diff(t) for q in Q]
#
#L = scene.get_energy_expr(t)
#
#display(Markdown('## Lagrangian:'))
#display(sp.Eq(sp.Symbol('L'), L))
#
#display(Markdown('## Lagrange Equations:'))
#eqs = [get_lagrange_eq(L, q, t) for q in Q]
#display({q.diff(t, 2): eq for q, eq in zip(Q, eqs)})

In [ ]:
#l1 = 3.
#l2 = 3.
#m1 = 1.
#m2 = 1.
#g = 10.
#def get_ddt2(t1, t2, dt1, dt2):
#    return 1/(m2*l2**2-1/(m1+m2)*(m2*l1*l2*np.cos(t2-t1))) * (m2*l1*l2*dt1*(dt2-dt1)*np.sin(t2-t1)-m2*g*np.cos(t2)-1/(m1+m2)*(m2*l1*l2*np.cos(t2-t1))*(m2*l1*l2*dt2*(dt2-dt1)*np.sin(t2-t1)-(m1+m2)*g*np.cos(t1)))
#
#def get_ddt1(t1, t2, dt1, dt2):
#    ddt2 = get_ddt2(t1, t2, dt1, dt2)
#    return 1/((m1+m2)*l1**2) * (m2*l1*l2*dt2*(dt2-dt1)*np.sin(t2-t1)-(m1+m2)*g*np.cos(t1) - m2*l1*l2*np.cos(t2-t1)*ddt2)
#
#funcs = [
#    #lambda theta1, theta2, dtheta1, dtheta2: m2*l2*np.sin(theta2-theta1)-GRAVITY/l1*np.sin(theta1),
#    #lambda theta1, theta2, dtheta1, dtheta2: -m2*l1*np.sin(theta2-theta1)-GRAVITY/l2*np.sin(theta2),
#    get_ddt1,
#    get_ddt2,
#]

In [ ]:
#eqs = solve_lagrangian(L, Q, t)

In [ ]:
#params = {symbol: default_value for symbol, default_value in scene.param_symbols}
#funcs = get_funcs(Q, eqs, params)

In [ ]:
world = World()
track = world | TrackSpec()
block = track | BlockSpec()
hinge = block | HingeSpec()
rod = hinge | RodSpec()
ball = rod | BallSpec()

scene = Scene([ball2])

@interact(
    track_x=(-2., 15.),
    hinge_theta=(-2*np.pi, 2*np.pi),
)
def f(
    track_x=2.,
    rod_length=3.,
    hinge_theta=0.5,
):
    params = {
        rod.spec.length: rod_length,
        world.spec.gravity: 10.,
        block.spec.mass: 1.,
        ball.spec.mass: 1.,
    }
    state = {
        hinge.spec.theta: hinge_theta,
        track.spec.x: track_x,
        hinge.spec.theta.diff(t): 0.,
        track.spec.x.diff(t): 0.,
    }
    fig, ax = plt.subplots()
    scene.draw(ax, params, state)

In [ ]:
def fig2np(fig):
    fig.canvas.draw() 
    w, h = fig.canvas.get_width_height()
    buffer = fig.canvas.buffer_rgba()
    return (
        np
        .frombuffer(buffer, dtype=np.uint8)
        .reshape(h, w, 4)
    )


fig, ax = plt.subplots(dpi=100)
for x_ in np.linspace(0., 2., 5):
    params = {}
    initial_state = {
        x: x_,
    }
    scene.draw(ax, params=params, state=initial_state, scale=scale)
    ax.axis('off')
    image = fig2np(fig)

    ax.clear()
    
fig.clear()

fig, ax2 = plt.subplots(figsize=(16*.8, 9*.8))
ax2.imshow(image, aspect='equal', interpolation='hanning')
ax2.axis('off')
None

In [ ]:
image.shape

In [ ]:
np.stack([image, image, image]).shape

In [ ]:
def sample_iter(iterator, sample_interval):
    return islice(iterator, 0, None, sample_interval)


with timed() as timing:
    sample_interval = 5
    frames = []
    with renderer_scope(frames) as frame_scope:
        iterator = simulator.iterate(params, initial_state, max_time=300)
        for state in sample_iter(iterator, sample_interval):
            with frame_scope() as ax:
                scene.draw(ax, params=params, state=state, scale=scale)

timing[0], len(frames)

In [ ]:
with timed() as timing:
    states = list(simulator.iterate(params, initial_state, max_time=300))
    draw_func = lambda ax, time_index: scene.draw(ax, params, states[time_index], scale=scale)
    anim = do_render_animation(draw_func, len(states), sample_interval)
    anim.save('anim.mp4')

#display(anim)
timing[0]

In [ ]:

@interact(t=(0, len(frames) - 1))
def f(t=0):
    fig, ax = plt.subplots(figsize=(16*.8, 9*.8))
    ax.axis('off')
    ax.imshow(frames[t], aspect='equal', interpolation='hanning')

In [ ]:
np.matrix([[2, 0], [0, 1]]) * np.matrix([[0, 1], [2, 3], [4, 5]]).T

In [ ]:
points = np.matrix([[0, 1], [2, 3], [4, 5]])

np.vstack([points, [[6, 7], [8, 9]]])



In [ ]:
points[1] * 3